## **앙상블 학습**

### 앙상블 학습을 통한 분류
- 여러 개의 분류기를 사용해서 예측 결합함으로써 보다 정확한 최종 예측을 도출하는 기법
- 단일 분류기 사용 때보다 신뢰성이 높은 예측값을 얻을 수 있음
- 쉽고 편하면서도 강력한 성능 보유
- 대부분의 정형 데이터 분류 시 뛰어난 성능을 나타냄
- 이미지, 영상, 음성 등의 비정형 데이터 분류 : 딥러닝 성능 뛰어남

### 대표적인 앙상블 알고리즘
- 랜덤 포레스트(Random Forrest)
- 그레디언트 부스팅(Gradient Boosting)

### 최신 앙상블 알고리즘
- XGBoost
- LightBGM : XGBoost와 예측 성능 유사하면서도 수행 속도 훨씬 빠름
- Stacking : 여러 가지 모델의 결과를 기반으로 메타 모델 수립

XGBoost,LightBGM과 같은 최신 앙상블 알고리즘 한두 개만 잘 알고 있어도 정형데이터의 분류 또는 회귀 분야에서 예측 성능이 매우 뛰어난 모델을 쉽게 만들 수 있음

### 앙상블 학습 유형
- 보팅(Voting)
- 배깅(Bagging) -> 랜덤포레스트
- 부스팅(Boosting)
- 스태킹(Stacking)

---

## **Voting Classifier**

### 보팅 방식의 앙상블 예제 : 위스콘신 유방암 데이터 세트 예측 분석

위스콘신 유방암 데이터 로드

In [8]:
import pandas as pd

from sklearn.ensemble import VotingClassifier # voting 분류기
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀
from sklearn.neighbors import KNeighborsClassifier # knn
from sklearn.datasets import load_breast_cancer # dataset
from sklearn.model_selection import train_test_split # train/test분류
from sklearn.metrics import accuracy_score # 성능평가
import warnings

warnings.filterwarnings('ignore')
cancer = load_breast_cancer()
data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


#### VotingClassifier로 개별모델은 로지스틱 회귀와 KNN을 보팅방식으로 결합하고 성능 비교

In [2]:
# 개별 모델은 로지스틱 회귀와 KNN
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 보팅 방식 : default -> hard
vo_clf = VotingClassifier(estimators=[('LR',lr_clf),('KNN',knn_clf)],voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                   test_size=0.2,random_state=156)

In [4]:
# VotingClassifier 학습/예측/평가
# 개별 모델들이 다 학습하고 예측한 결과로 평가
vo_clf.fit(X_train,y_train)
pred = vo_clf.predict(X_test)
print('보팅을 통한 분류기 정확도:{0:.4f}'.format(accuracy_score(y_test,pred)))

보팅을 통한 분류기 정확도:0.9474


C:\ananconda\envs\multi\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [7]:
# 로지스틱 회귀와 KNN 각각 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]

for classifier in classifiers:
    classifier.fit(X_train,y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print('{0} 정확도:{1:.4f}'.format(class_name, accuracy_score(y_test,pred)))

LogisticRegression 정확도:0.9386
KNeighborsClassifier 정확도:0.9386


C:\ananconda\envs\multi\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


---

## **랜덤 포레스트**
- 기반 알고리즘은 의사결정
- 배깅방식
- 부트스트래핑 분할방식

In [9]:
# 중복된 피처변경/반환하는 함수 (전처리)
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df


In [24]:
# 사용자 행동 인식 데이터 세트 준비함수
def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('../features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 feature명을 새롭게 수정하는 get_new_feature_name_df()를 이용하여 새로운 feature명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('../train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('../test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('../train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('../test/y_test.txt',sep='\s+',header=None,names=['action'])

    # 로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test


### 학습/테스트 데이터로 분리하고 랜덤 포레스트로 학습/예측/평가

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train,y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test,pred)
print('랜덤 포레스트 정확도:{0:.4f}'.format(accuracy))

랜덤 포레스트 정확도:0.9253


### 트리 기반의 앙상블 알고리즘의 단점
- 하이퍼파라미터가 너무 많아 그로 인해 튜닝을 위한 시간이 많이 소모

In [31]:
## 하이퍼파라미터 튜닝과정
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth' : [6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split' : [8,16,20]
}

rf_clf = RandomForestClassifier(random_state=0,n_jobs=-1)

grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터 :\n', grid_cv.best_params_)
print('최적 예측 정확도 : {0:.4f}'.format(grid_cv.best_score_))


UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-19: ordinal not in range(128)

### 개별 feature들의 중요도 시각화

In [33]:
# import seaborn as sns
# ftr_importances_values = rf_clf.feature_importances_
# ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns)

# ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
# plt.figure(figsize=(8,6))
# plt.title('Feature importance Top 20')
# sns.barplot(x=ftr_top20, y=ftr_top20.index)
# plt.show()

---

## GBM(Gradient Boosting Machine)

### 부스팅(Boosting)
- 여러 개의 약한 학습기를 순차적으로 학습-예측하면서
- 잘못 예측된 데이터에 가중치부여를 통해
- 오류를 개선해 나가면서 학습하는 방식

### 대표적 부스팅 알고리즘
- AdaBoost : 에이다 부스트
- GBM : 그래디언트 부스트

In [32]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train , y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

GBM 정확도: 0.9389
GBM 수행 시간: 1143.4 초 


### GridSearchCV 이용해서 하이퍼 파라미터 최적화

In [ ]:
# from sklearn.model_selection import GridSearchCV

# params = {
#     'n_estimators':[100, 500],
#     'learning_rate' : [ 0.05, 0.1]
# }
# grid_cv = GridSearchCV(gb_clf , param_grid=params , cv=2 ,verbose=1)
# grid_cv.fit(X_train , y_train)
# print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
# print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))


GBM은 수행 시간이 오래 걸린다는 단점이 있지만 과적합에도 강해서 예측 성능이 뛰어난 알고리즘
- XGBoost
- LightGBM